In [ ]:
%run /home/jovyan/work/testing.py

✅ Bucket created!
✅ Job completed!

Files in bucket:
  - output.parquet/_SUCCESS
  - output.parquet/part-00000-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00001-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00002-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00003-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00004-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00005-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00006-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00007-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00008-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00009-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00010-c1138f74-f87f-4e02-

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TestS3") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✅ Spark is ready!")

✅ Spark is ready!


In [6]:
# Read back the data
df_read = spark.read.parquet("s3a://test-bucket/output.parquet")
df_read.show()

+---+-------+
| id|doubled|
+---+-------+
| 16|     32|
| 17|     34|
| 18|     36|
| 19|     38|
| 20|     40|
| 21|     42|
| 22|     44|
| 23|     46|
| 24|     48|
| 41|     82|
| 42|     84|
| 43|     86|
| 44|     88|
| 45|     90|
| 46|     92|
| 47|     94|
| 48|     96|
| 49|     98|
| 66|    132|
| 67|    134|
+---+-------+
only showing top 20 rows



### Kafka spark



In [1]:
!pip install boto3
!pip install requests

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/51/4a/5d33b6046d425c9b39d36a1171ea87a9c3b297ba116952b81033eae61260/boto3-1.40.11-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.41.0,>=1.40.11 from https://files.pythonhosted.org/packages/2d/f9/400e0da61cbbcea7868458f3a447d1191a62ae5e2852d2acdfd4d51b2843/botocore-1.40.11-py3-none-any.whl.metadata
  Obtaining dependency information for jmespath<2.0.0,>=0.7.1 from https://files.pythonhosted.org/packages/31/b4/b9b800c45527aadd64d5b442f9b932b00648617eb5d63d2c7a6587b7cafc/jmespath-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.14.0,>=0.13.0 from https://files.pythonhosted.org/packages/6d/4f/d073e09df851cfa251ef7840007d04db3293a0482ce607d2b993926089be/s3transfer-0.13.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 557.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB

In [2]:

import boto3
s3 = boto3.client('s3', endpoint_url='http://localstack:4566', 
                  aws_access_key_id='test', aws_secret_access_key='test')
s3.create_bucket(Bucket='test-bucket')
print("Bucket created!")

KeyboardInterrupt: 

In [3]:
!pip install kafka-python


  Obtaining dependency information for kafka-python from https://files.pythonhosted.org/packages/e6/35/e8bfed5425e8fe685bd03ec3f5135ee8b88c11558baa59c0d12fbd2a20ae/kafka_python-2.2.15-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.8/309.8 kB 1.6 MB/s eta 0:00:00a 0:00:01m


In [4]:

from kafka.admin import KafkaAdminClient, NewTopic

admin = KafkaAdminClient(bootstrap_servers='kafka:29092')
topic = NewTopic(name='test-topic', num_partitions=1, replication_factor=1)
admin.create_topics([topic])
print("Topic created!")

Topic created!


In [5]:
%run /home/jovyan/work/simple_producer.py

Sent: {'id': 0, 'name': 'Message 0', 'value': 0}
Sent: {'id': 1, 'name': 'Message 1', 'value': 100}
Sent: {'id': 2, 'name': 'Message 2', 'value': 200}
Sent: {'id': 3, 'name': 'Message 3', 'value': 300}
Sent: {'id': 4, 'name': 'Message 4', 'value': 400}
Sent: {'id': 5, 'name': 'Message 5', 'value': 500}
Sent: {'id': 6, 'name': 'Message 6', 'value': 600}
Sent: {'id': 7, 'name': 'Message 7', 'value': 700}
Sent: {'id': 8, 'name': 'Message 8', 'value': 800}
Sent: {'id': 9, 'name': 'Message 9', 'value': 900}
Done sending messages!


In [6]:
%run /home/jovyan/work/simple_spark_consumer.py

✅ Spark session created
📖 Reading from Kafka...

📊 Data from Kafka:
+---+---------+-----+
| id|     name|value|
+---+---------+-----+
|  0|Message 0|    0|
|  1|Message 1|  100|
|  2|Message 2|  200|
|  3|Message 3|  300|
|  4|Message 4|  400|
|  5|Message 5|  500|
|  6|Message 6|  600|
|  7|Message 7|  700|
|  8|Message 8|  800|
|  9|Message 9|  900|
+---+---------+-----+


💾 Writing to Delta Lake at: s3a://test-bucket/delta-tables/my_table
✅ Data saved to Delta Lake!

📖 Reading back from Delta Lake:
+---+---------+-----+
| id|     name|value|
+---+---------+-----+
|  0|Message 0|    0|
|  1|Message 1|  100|
|  2|Message 2|  200|
|  3|Message 3|  300|
|  4|Message 4|  400|
|  5|Message 5|  500|
|  6|Message 6|  600|
|  7|Message 7|  700|
|  8|Message 8|  800|
|  9|Message 9|  900|
+---+---------+-----+


✅ Test completed successfully!


In [4]:
import boto3

# Configure boto3 for LocalStack
s3_client = boto3.client(
    's3',
    endpoint_url='http://localstack:4566',
    aws_access_key_id='test',
    aws_secret_access_key='test',
    region_name='us-east-1'
)

# List objects in the Delta table path
try:
    response = s3_client.list_objects_v2(Bucket='test-bucket', Prefix='delta-table/')
    if 'Contents' in response:
        print("Files in s3://test-bucket/delta-table/:")
        for obj in response['Contents']:
            print(f" - {obj['Key']} ({obj['Size']} bytes)")
    else:
        print("No files found in s3://test-bucket/delta-table/ or bucket does not exist.")
except s3_client.exceptions.NoSuchBucket:
    print("Bucket 'test-bucket' does not exist. Creating it...")
    s3_client.create_bucket(Bucket='test-bucket')
    print("Bucket created. Rerun the Spark job to write the Delta table.")
except Exception as e:
    print(f"Error: {str(e)}")

Files in s3://test-bucket/delta-table/:
 - delta-table/_delta_log/00000000000000000000.json (1248 bytes)
 - delta-table/part-00000-354a5e3f-e191-438b-bbac-aec33463ff8b-c000.snappy.parquet (1177 bytes)


In [ ]:
# Create Spark session with Delta support
spark = SparkSession.builder \
    .appName("SimpleTest") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
output_path = "s3a://test-bucket/delta-table"
print("\n📖 Reading back from Delta Lake:")
delta_df = spark.read.format("delta").load(output_path)
delta_df.show()


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create Spark session with Delta support
spark = SparkSession.builder \
    .appName("SimpleTest") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✅ Spark session created")

# Read from Kafka
print("📖 Reading from Kafka...")
df = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "test-topic") \
    .option("startingOffsets", "earliest") \
    .load()

# Parse JSON and select fields
parsed_df = df.select(
    col("value").cast("string")
).select(
    get_json_object(col("value"), "$.id").alias("id"),
    get_json_object(col("value"), "$.name").alias("name"),
    get_json_object(col("value"), "$.value").alias("value")
)

# Show data
print("\n📊 Data from Kafka:")
parsed_df.show()

# Write to Delta Lake on S3
#output_path = "s3a://test-bucket/delta-table"
output_path = "s3a://test-bucket/delta-tables/my_table"
print(f"\n💾 Writing to Delta Lake at: {output_path}")
parsed_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save(output_path)

print("✅ Data saved to Delta Lake!")

# Read back from Delta to verify
print("\n📖 Reading back from Delta Lake:")
delta_df = spark.read.format("delta").load(output_path)
delta_df.show()

print("\n✅ Test completed successfully!")
spark.stop()

✅ Spark session created
📖 Reading from Kafka...

📊 Data from Kafka:
+---+---------+-----+
| id|     name|value|
+---+---------+-----+
|  0|Message 0|    0|
|  1|Message 1|  100|
|  2|Message 2|  200|
|  3|Message 3|  300|
|  4|Message 4|  400|
|  5|Message 5|  500|
|  6|Message 6|  600|
|  7|Message 7|  700|
|  8|Message 8|  800|
|  9|Message 9|  900|
+---+---------+-----+


💾 Writing to Delta Lake at: s3a://test-bucket/delta-tables/my_table
✅ Data saved to Delta Lake!

📖 Reading back from Delta Lake:
+---+---------+-----+
| id|     name|value|
+---+---------+-----+
|  0|Message 0|    0|
|  1|Message 1|  100|
|  2|Message 2|  200|
|  3|Message 3|  300|
|  4|Message 4|  400|
|  5|Message 5|  500|
|  6|Message 6|  600|
|  7|Message 7|  700|
|  8|Message 8|  800|
|  9|Message 9|  900|
+---+---------+-----+


✅ Test completed successfully!


In [8]:
%run /home/jovyan/work/prod.py

✅ Sent: {'symbol': 'GOOGL', 'price': 289.41, 'volume': 877, 'timestamp': '2025-08-04T15:39:32.196996', 'exchange': 'XNAS'}
✅ Sent: {'symbol': 'GOOGL', 'price': 298.55, 'volume': 36, 'timestamp': '2025-08-04T15:39:33.355669', 'exchange': 'XNAS'}
✅ Sent: {'symbol': 'MSFT', 'price': 275.08, 'volume': 850, 'timestamp': '2025-08-04T15:39:34.356757', 'exchange': 'XNAS'}
✅ Sent: {'symbol': 'GOOGL', 'price': 205.54, 'volume': 737, 'timestamp': '2025-08-04T15:39:35.358286', 'exchange': 'XNAS'}
✅ Sent: {'symbol': 'TSLA', 'price': 152.65, 'volume': 780, 'timestamp': '2025-08-04T15:39:36.359188', 'exchange': 'XNAS'}
✅ Sent: {'symbol': 'TSLA', 'price': 246.13, 'volume': 401, 'timestamp': '2025-08-04T15:39:37.359791', 'exchange': 'XNAS'}
✅ Sent: {'symbol': 'MSFT', 'price': 209.88, 'volume': 114, 'timestamp': '2025-08-04T15:39:38.360518', 'exchange': 'XNAS'}
✅ Sent: {'symbol': 'AAPL', 'price': 189.29, 'volume': 299, 'timestamp': '2025-08-04T15:39:39.361496', 'exchange': 'XNAS'}
✅ Sent: {'symbol': 'AA

In [5]:
%run /home/jovyan/work/stock_consumer.py

✅ Spark session created


NameError: name 'StructField' is not defined

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, expr
from pyspark.sql.types import StructType, StringType, FloatType, IntegerType
import requests
import time

# Exchange → City mapping
exchange_city_map = {
    "XNAS": "New York",
    "XNYS": "New York", 
    "XLON": "London",
    "XTKS": "Tokyo"
}

# Cache for weather per city
weather_cache = {}
CACHE_TTL_SECONDS = 600  # 10 minutes

def fetch_weather_cached(city):
    now = time.time()
    cached = weather_cache.get(city)

    if cached and now - cached["fetched_at"] < CACHE_TTL_SECONDS:
        return cached["data"]

    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid=34debbcec75c0e6f7351fcda60b141be&units=metric"
        response = requests.get(url, timeout=5)
        data = response.json()
        weather = {
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "condition": data["weather"][0]["description"]
        }
        # Save to cache
        weather_cache[city] = {
            "fetched_at": now,
            "data": weather
        }
        return weather
    except Exception as e:
        print(f"⚠️ Weather fetch failed for {city}: {e}")
        return {"temperature": None, "humidity": None, "condition": None}

# Initialize Spark
spark = SparkSession.builder \
    .appName("StockWeatherEnrichment") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("✅ Spark session created")

# CORRECT schema matching your producer data
schema = StructType() \
    .add("symbol", StringType()) \
    .add("price", FloatType()) \
    .add("volume", IntegerType()) \
    .add("timestamp", StringType()) \
    .add("exchange", StringType())

print("📖 Reading from Kafka...")

# Read from Kafka (batch mode for testing)
df = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "test-topic") \
    .option("startingOffsets", "earliest") \
    .load()

# First, let's see the raw data
print("\n🔍 Raw Kafka data:")
df.selectExpr("CAST(value AS STRING) as raw_value").show(5, truncate=False)

# Parse JSON with correct schema
parsed_df = df.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json("json_str", schema).alias("data")) \
    .select("data.*")

print("\n📊 Parsed data:")
parsed_df.show()

# Test enrichment function
def enrich_with_weather_batch(input_df):
    if input_df.count() == 0:
        print("⚠️ No data to process")
        return input_df
    
    print(f"📦 Processing {input_df.count()} records")
    
    # Get unique exchanges
    exchanges = [row['exchange'] for row in input_df.select("exchange").distinct().collect()]
    print(f"🌍 Exchanges found: {exchanges}")
    
    # Fetch weather for each exchange
    weather_data = {}
    for ex in exchanges:
        city = exchange_city_map.get(ex, "New York")
        print(f"🌤️ Fetching weather for {ex} ({city})")
        weather_data[ex] = fetch_weather_cached(city)
    
    # Add weather columns using broadcast
    from pyspark.sql.functions import lit, when
    
    enriched_df = input_df
    
    # Add weather columns based on exchange
    for exchange, weather in weather_data.items():
        enriched_df = enriched_df \
            .withColumn("temperature", 
                when(col("exchange") == exchange, lit(weather.get("temperature")))
                .otherwise(col("temperature") if "temperature" in enriched_df.columns else lit(None))) \
            .withColumn("humidity",
                when(col("exchange") == exchange, lit(weather.get("humidity")))
                .otherwise(col("humidity") if "humidity" in enriched_df.columns else lit(None))) \
            .withColumn("condition",
                when(col("exchange") == exchange, lit(weather.get("condition")))
                .otherwise(col("condition") if "condition" in enriched_df.columns else lit(None)))
    
    return enriched_df

# Process the data
if parsed_df.count() > 0:
    enriched_df = enrich_with_weather_batch(parsed_df)
    
    print("\n🌟 Enriched data:")
    enriched_df.show()
    
    # Write to Delta Lake
    print("\n💾 Writing to Delta Lake...")
    enriched_df.write.format("delta") \
        .mode("overwrite") \
        .save("s3a://test-bucket/delta-tables/enriched_trades")
    
    print("✅ Data written to Delta Lake!")
    
    # Read back to verify
    print("\n📖 Reading back from Delta Lake:")
    delta_df = spark.read.format("delta").load("s3a://test-bucket/delta-tables/enriched_trades")
    delta_df.show()
    
else:
    print("⚠️ No data found in Kafka topic")

print("✅ Processing completed!")

✅ Spark session created
📖 Reading from Kafka...

🔍 Raw Kafka data:
+------------------------------------------------------------------------------------------------------------------+
|raw_value                                                                                                         |
+------------------------------------------------------------------------------------------------------------------+
|{"symbol": "MSFT", "price": 154.31, "volume": 622, "timestamp": "2025-08-04T17:39:06.433532", "exchange": "XNAS"} |
|{"symbol": "GOOGL", "price": 188.03, "volume": 656, "timestamp": "2025-08-04T17:39:07.545608", "exchange": "XNAS"}|
|{"symbol": "TSLA", "price": 174.87, "volume": 519, "timestamp": "2025-08-04T17:39:08.546064", "exchange": "XNAS"} |
|{"symbol": "AAPL", "price": 166.18, "volume": 881, "timestamp": "2025-08-04T17:39:09.546507", "exchange": "XNAS"} |
|{"symbol": "TSLA", "price": 168.09, "volume": 163, "timestamp": "2025-08-04T17:39:10.547317", "exchange": "XNAS"}

In [11]:
# streaming_processor.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, lit, when, date_format, avg, sum as spark_sum, count
from pyspark.sql.types import StructType, StringType, FloatType, IntegerType
import requests
import time

# Weather enrichment
exchange_city_map = {
    "XNAS": "New York",
    "XNYS": "New York",
    "XLON": "London",
    "XTKS": "Tokyo"
}

weather_cache = {}
CACHE_TTL_SECONDS = 600

def fetch_weather_cached(city):
    now = time.time()
    cached = weather_cache.get(city)
    
    if cached and now - cached["fetched_at"] < CACHE_TTL_SECONDS:
        return cached["data"]
    
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid=34debbcec75c0e6f7351fcda60b141be&units=metric"
        response = requests.get(url, timeout=5)
        data = response.json()
        weather = {
            "temperature": float(data["main"]["temp"]),
            "humidity": int(data["main"]["humidity"]),
            "condition": data["weather"][0]["description"]
        }
        weather_cache[city] = {"fetched_at": now, "data": weather}
        return weather
    except Exception as e:
        print(f"⚠️ Weather fetch failed for {city}: {e}")
        return {"temperature": None, "humidity": None, "condition": "unknown"}

# Initialize Spark
spark = SparkSession.builder \
    .appName("FinTechStreamingPipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("✅ Spark Streaming session created")

# Schema for incoming data
schema = StructType() \
    .add("symbol", StringType()) \
    .add("price", FloatType()) \
    .add("volume", IntegerType()) \
    .add("timestamp", StringType()) \
    .add("exchange", StringType())

print("🌊 Starting streaming from Kafka...")

# STREAMING: Read from Kafka continuously
streaming_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "test-topic") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load()

# Parse streaming data
parsed_stream = streaming_df.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json("json_str", schema).alias("data")) \
    .select("data.*") \
    .withColumn("processing_time", lit(int(time.time())))

# Enrichment function for streaming batches
def enrich_and_store_stream(batch_df, batch_id):
    if batch_df.count() == 0:
        return
    
    print(f"🌊 Processing streaming batch {batch_id} with {batch_df.count()} records")
    
    # Get weather data for unique exchanges
    exchanges = [row['exchange'] for row in batch_df.select("exchange").distinct().collect()]
    weather_data = {}
    
    for ex in exchanges:
        city = exchange_city_map.get(ex, "New York")
        print(f"🌤️ Fetching weather for {ex} ({city})")
        weather_data[ex] = fetch_weather_cached(city)
    
    # ✅ SIMPLIFIED: Only add weather columns (no city)
    enriched_df = batch_df
    for col_name in ["temperature", "humidity", "condition"]:
        enriched_df = enriched_df.withColumn(col_name, lit(None))
    
    for exchange, weather in weather_data.items():
        enriched_df = enriched_df \
            .withColumn("temperature", when(col("exchange") == exchange, lit(weather.get("temperature"))).otherwise(col("temperature"))) \
            .withColumn("humidity", when(col("exchange") == exchange, lit(weather.get("humidity"))).otherwise(col("humidity"))) \
            .withColumn("condition", when(col("exchange") == exchange, lit(weather.get("condition"))).otherwise(col("condition")))
    
    # ✅ ADD DATE PARTITIONING COLUMN
    enriched_df = enriched_df \
        .withColumn("date", date_format(col("timestamp"), "yyyy-MM-dd"))
    
    # ✅ WRITE WITH DATE PARTITIONING ONLY
    enriched_df.write.format("delta") \
        .mode("append") \
        .partitionBy("date") \
        .save("s3a://test-bucket/delta-tables/streaming_trades")
    
    print(f"✅ Batch {batch_id} written to streaming storage with date partitioning")

# Start the streaming query
streaming_query = parsed_stream.writeStream \
    .foreachBatch(enrich_and_store_stream) \
    .outputMode("append") \
    .trigger(processingTime='5 seconds') \
    .option("checkpointLocation", "s3a://test-bucket/checkpoints/streaming") \
    .start()

print("🌊 Streaming query started! Processing every 5 seconds...")
print("💡 Run the producer in another terminal to see real-time processing")
print("🌍 Weather data will be fetched for multiple cities based on exchanges")

# Keep streaming running
try:
    streaming_query.awaitTermination()
except KeyboardInterrupt:
    print("\n🛑 Stopping streaming...")
    streaming_query.stop()
    print("✅ Streaming stopped")

✅ Spark Streaming session created
🌊 Starting streaming from Kafka...
🌊 Streaming query started! Processing every 5 seconds...
💡 Run the producer in another terminal to see real-time processing
🌍 Weather data will be fetched for multiple cities based on exchanges
🌊 Processing streaming batch 5 with 10 records
🌤️ Fetching weather for XNYS (New York)
🌤️ Fetching weather for XLON (London)
✅ Batch 5 written to streaming storage with date partitioning
🌊 Processing streaming batch 6 with 15 records
🌤️ Fetching weather for XLON (London)
🌤️ Fetching weather for XNAS (New York)
🌤️ Fetching weather for XNYS (New York)
✅ Batch 6 written to streaming storage with date partitioning
🌊 Processing streaming batch 7 with 10 records
🌤️ Fetching weather for XLON (London)
🌤️ Fetching weather for XNYS (New York)
🌤️ Fetching weather for XNAS (New York)
✅ Batch 7 written to streaming storage with date partitioning
🌊 Processing streaming batch 8 with 5 records
🌤️ Fetching weather for XLON (London)
🌤️ Fetching 

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt



🛑 Stopping streaming...
✅ Streaming stopped


In [9]:
    print("\n📖 Reading back from Delta Lake:")
    delta_df = spark.read.format("delta").load("s3a://test-bucket/delta-tables/streaming_trades")
    delta_df.show()


📖 Reading back from Delta Lake:
+------+------+------+--------------------+--------+---------------+-----------+--------+---------------+----------+
|symbol| price|volume|           timestamp|exchange|processing_time|temperature|humidity|      condition|      date|
+------+------+------+--------------------+--------+---------------+-----------+--------+---------------+----------+
|   JPM|106.87|   777|2025-08-06T20:27:...|    XNYS|     1754512047|      23.99|      63|          smoke|2025-08-06|
|  TSLA|285.96|   684|2025-08-06T20:27:...|    XNAS|     1754512047|      23.99|      63|          smoke|2025-08-06|
|  AAPL| 213.2|   125|2025-08-06T20:27:...|    XNAS|     1754512047|      23.99|      63|          smoke|2025-08-06|
|   XOM|132.37|   354|2025-08-06T20:27:...|    XNYS|     1754512047|      23.99|      63|          smoke|2025-08-06|
|  TSLA|205.22|   795|2025-08-06T20:27:...|    XNAS|     1754512047|      23.99|      63|          smoke|2025-08-06|
|  TSLA|124.18|   654|2025-08-0

In [10]:
# batch_processor.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, avg, sum as spark_sum, count, max as spark_max, min as spark_min, lit
from datetime import datetime, timedelta

# Initialize Spark
spark = SparkSession.builder \
    .appName("FinTechBatchProcessor") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("✅ Spark Batch session created")

def run_batch_aggregations(target_date=None):
    """
    Run batch aggregations for a specific date.
    If no date specified, process all available data.
    """
    print("📊 Starting batch processing...")
    
    # Read streaming data
    streaming_data = spark.read.format("delta") \
        .load("s3a://test-bucket/delta-tables/streaming_trades")
    
    # Filter by specific date if provided
    if target_date:
        streaming_data = streaming_data.filter(col("date") == target_date)
        print(f"📅 Processing data for date: {target_date}")
    
    if streaming_data.count() == 0:
        print("⚠️ No streaming data found for batch processing")
        return
    
    print(f"📈 Processing {streaming_data.count()} records from streaming storage")
    
    # ✅ SIMPLIFIED HOURLY AGGREGATIONS (no city column)
    hourly_summary = streaming_data \
        .withColumn("hour", date_format(col("timestamp"), "yyyy-MM-dd HH")) \
        .groupBy("symbol", "exchange", "hour", "date", "condition") \
        .agg(
            avg("price").alias("avg_price"),
            spark_min("price").alias("min_price"),
            spark_max("price").alias("max_price"),
            spark_sum("volume").alias("total_volume"),
            count("*").alias("trade_count"),
            avg("temperature").alias("avg_temperature"),
            avg("humidity").alias("avg_humidity")
        ) \
        .withColumn("batch_processed_at", lit(datetime.now().isoformat()))
    
    print("📊 Hourly aggregations:")
    hourly_summary.show(10)
    
    # ✅ WRITE WITH DATE PARTITIONING AND PROPER MODE
    if target_date:
        # Replace only specific date
        hourly_summary.write.format("delta") \
            .mode("overwrite") \
            .option("replaceWhere", f"date = '{target_date}'") \
            .save("s3a://test-bucket/delta-tables/hourly_summaries")
    else:
        # Replace all data
        hourly_summary.write.format("delta") \
            .mode("overwrite") \
            .partitionBy("date") \
            .save("s3a://test-bucket/delta-tables/hourly_summaries")
    
    # ✅ SIMPLIFIED DAILY AGGREGATIONS (no city column)
    daily_summary = streaming_data \
        .groupBy("symbol", "exchange", "date") \
        .agg(
            avg("price").alias("avg_price"),
            spark_min("price").alias("min_price"),
            spark_max("price").alias("max_price"),
            spark_sum("volume").alias("total_volume"),
            count("*").alias("trade_count"),
            avg("temperature").alias("avg_temperature"),
            avg("humidity").alias("avg_humidity")
        ) \
        .withColumn("batch_processed_at", lit(datetime.now().isoformat()))
    
    print("📅 Daily aggregations:")
    daily_summary.show(10)
    
    # ✅ WRITE WITH DATE PARTITIONING
    if target_date:
        # Replace only specific date
        daily_summary.write.format("delta") \
            .mode("overwrite") \
            .option("replaceWhere", f"date = '{target_date}'") \
            .save("s3a://test-bucket/delta-tables/daily_summaries")
    else:
        # Replace all data
        daily_summary.write.format("delta") \
            .mode("overwrite") \
            .partitionBy("date") \
            .save("s3a://test-bucket/delta-tables/daily_summaries")
    
    print("✅ Batch processing completed with date partitioning!")

def run_incremental_batch():
    """
    Process only yesterday's data (typical production scenario)
    """
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    print(f"📅 Running incremental batch for {yesterday}")
    run_batch_aggregations(target_date=yesterday)

if __name__ == "__main__":
    # Run for all data
    run_batch_aggregations()
    
    # Or run incremental (uncomment this line)
    # run_incremental_batch()

✅ Spark Batch session created
📊 Starting batch processing...
📈 Processing 19 records from streaming storage
📊 Hourly aggregations:
+------+--------+-------------+----------+---------------+------------------+---------+---------+------------+-----------+---------------+------------+--------------------+
|symbol|exchange|         hour|      date|      condition|         avg_price|min_price|max_price|total_volume|trade_count|avg_temperature|avg_humidity|  batch_processed_at|
+------+--------+-------------+----------+---------------+------------------+---------+---------+------------+-----------+---------------+------------+--------------------+
|  HSBA|    XLON|2025-08-06 20|2025-08-06|overcast clouds| 274.6000061035156|    274.6|    274.6|         608|          1|          19.19|        60.0|2025-08-06T20:29:...|
|   JPM|    XNYS|2025-08-06 20|2025-08-06|          smoke|106.87000274658203|   106.87|   106.87|         777|          1|          23.99|        63.0|2025-08-06T20:29:...|
| GO

separated hourly and daily 

In [16]:
# scripts/daily_processor.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, avg, sum as spark_sum, count, max as spark_max, min as spark_min, lit
from datetime import datetime, timedelta
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_spark_session():
    return SparkSession.builder \
        .appName("FinTechDailyProcessor") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.jars.packages", 
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
                "io.delta:delta-core_2.12:2.4.0,"
                "org.apache.hadoop:hadoop-aws:3.3.4,"
                "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
        .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
        .config("spark.hadoop.fs.s3a.access.key", "test") \
        .config("spark.hadoop.fs.s3a.secret.key", "test") \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .getOrCreate()

def run_daily_processing():
    """
    Process yesterday's complete day for daily summaries.
    """
    spark = create_spark_session()
    spark.sparkContext.setLogLevel("WARN")
    
    try:
        # Calculate yesterday
        yesterday = (datetime.now() - timedelta(days=0)).strftime("%Y-%m-%d")
        
        logger.info(f"📅 Processing daily data for: {yesterday}")
        
        # Read all streaming data for yesterday
        streaming_data = spark.read.format("delta") \
            .load("s3a://test-bucket/delta-tables/streaming_trades") \
            .filter(col("date") == yesterday)
        
        record_count = streaming_data.count()
        
        if record_count == 0:
            logger.warning(f"⚠️ No streaming data found for date: {yesterday}")
            return
        
        logger.info(f"📈 Processing {record_count} records for date: {yesterday}")
        
        # Create daily aggregation
        daily_summary = streaming_data \
            .groupBy("symbol", "exchange", "date") \
            .agg(
                avg("price").alias("avg_price"),
                spark_min("price").alias("min_price"),
                spark_max("price").alias("max_price"),
                spark_sum("volume").alias("total_volume"),
                count("*").alias("trade_count"),
                avg("temperature").alias("avg_temperature"),
                avg("humidity").alias("avg_humidity")
            ) \
            .withColumn("batch_processed_at", lit(datetime.now().isoformat()))
        
        daily_count = daily_summary.count()
        logger.info(f"📅 Created {daily_count} daily summary records")
        
        if daily_count > 0:
            # Replace only yesterday's daily summary
            daily_summary.write.format("delta") \
                .mode("overwrite") \
                .option("replaceWhere", f"date = '{yesterday}'") \
                .save("s3a://test-bucket/delta-tables/daily_summaries")
            
            logger.info(f"✅ Daily summary updated for {yesterday}")
        
        logger.info(f"🎉 Daily processing completed for {yesterday}!")
        
    except Exception as e:
        logger.error(f"❌ Daily processing failed: {str(e)}")
        raise
    finally:
        spark.stop()

if __name__ == "__main__":
    run_daily_processing()

In [13]:
# scripts/hourly_processor.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, avg, sum as spark_sum, count, max as spark_max, min as spark_min, lit
from datetime import datetime, timedelta
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_spark_session():
    return SparkSession.builder \
        .appName("FinTechHourlyProcessor") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.jars.packages", 
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
                "io.delta:delta-core_2.12:2.4.0,"
                "org.apache.hadoop:hadoop-aws:3.3.4,"
                "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
        .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
        .config("spark.hadoop.fs.s3a.access.key", "test") \
        .config("spark.hadoop.fs.s3a.secret.key", "test") \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .getOrCreate()

def run_hourly_processing():
    """
    Process the previous completed hour.
    E.g., if it's 10:05, process 09:00-09:59
    """
    spark = create_spark_session()
    spark.sparkContext.setLogLevel("WARN")
    
    try:
        # Calculate previous hour
        current_time = datetime.now()
        previous_hour = current_time.replace(minute=0, second=0, microsecond=0) #- timedelta(hours=1)
        target_hour = previous_hour.strftime("%Y-%m-%d %H")
        target_date = previous_hour.strftime("%Y-%m-%d")
        
        logger.info(f"🕐 Processing hourly data for: {target_hour}")
        
        # Read streaming data for the specific hour
        streaming_data = spark.read.format("delta") \
            .load("s3a://test-bucket/delta-tables/streaming_trades") \
            .filter(col("date") == target_date) \
            .filter(date_format(col("timestamp"), "yyyy-MM-dd HH") == target_hour)
        
        record_count = streaming_data.count()
        
        if record_count == 0:
            logger.warning(f"⚠️ No streaming data found for hour: {target_hour}")
            return
        
        logger.info(f"📈 Processing {record_count} records for hour: {target_hour}")
        
        # Create hourly aggregation
        hourly_summary = streaming_data \
            .withColumn("hour", lit(target_hour)) \
            .groupBy("symbol", "exchange", "hour", "date", "condition") \
            .agg(
                avg("price").alias("avg_price"),
                spark_min("price").alias("min_price"),
                spark_max("price").alias("max_price"),
                spark_sum("volume").alias("total_volume"),
                count("*").alias("trade_count"),
                avg("temperature").alias("avg_temperature"),
                avg("humidity").alias("avg_humidity")
            ) \
            .withColumn("batch_processed_at", lit(datetime.now().isoformat()))
        
        hourly_count = hourly_summary.count()
        logger.info(f"📊 Created {hourly_count} hourly summary records")
        
        if hourly_count > 0:
            # Use merge/upsert to replace only this specific hour
            hourly_summary.write.format("delta") \
                .mode("overwrite") \
                .option("replaceWhere", f"hour = '{target_hour}'") \
                .save("s3a://test-bucket/delta-tables/hourly_summaries")
            
            logger.info(f"✅ Hourly summary updated for {target_hour}")
        
        logger.info(f"🎉 Hourly processing completed for {target_hour}!")
        
    except Exception as e:
        logger.error(f"❌ Hourly processing failed: {str(e)}")
        raise
    finally:
        spark.stop()

if __name__ == "__main__":
    run_hourly_processing()

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, lit, when, date_format, avg, sum as spark_sum, count
from pyspark.sql.types import StructType, StringType, FloatType, IntegerType
import requests
import time

spark = SparkSession.builder \
    .appName("FinTechStreamingPipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
print("\n📖 Reading back from Delta Lake:")
delta_df = spark.read.format("delta").load("s3a://test-bucket/delta-tables/hourly_summaries")
delta_df.show()


📖 Reading back from Delta Lake:
+------+--------+-------------+----------+---------------+------------------+---------+---------+------------+-----------+------------------+------------+--------------------+
|symbol|exchange|         hour|      date|      condition|         avg_price|min_price|max_price|total_volume|trade_count|   avg_temperature|avg_humidity|  batch_processed_at|
+------+--------+-------------+----------+---------------+------------------+---------+---------+------------+-----------+------------------+------------+--------------------+
|   BAC|    XNYS|2025-08-16 15|2025-08-16|  broken clouds|224.82499313354492|   166.53|   288.61|        1404|          4|             26.89|        68.0|2025-08-16T15:24:...|
|  MSFT|    XNAS|2025-08-16 15|2025-08-16|  broken clouds|196.22000122070312|   156.67|   225.38|        1067|          3|             26.89|        68.0|2025-08-16T15:24:...|
|    BP|    XLON|2025-08-16 15|2025-08-16|overcast clouds| 278.0850067138672|   273.64|

In [6]:
# streaming_processor.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, lit, when, date_format, to_timestamp
from pyspark.sql.types import StructType, StringType, FloatType, IntegerType, TimestampType
import requests, time

# Weather enrichment (unchanged)
exchange_city_map = {
    "XNAS": "New York",
    "XNYS": "New York",
    "XLON": "London",
    "XTKS": "Tokyo"
}

weather_cache = {}
CACHE_TTL_SECONDS = 600

def fetch_weather_cached(city):
    now = time.time()
    cached = weather_cache.get(city)
    
    if cached and now - cached["fetched_at"] < CACHE_TTL_SECONDS:
        return cached["data"]
    
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid=34debbcec75c0e6f7351fcda60b141be&units=metric"
        response = requests.get(url, timeout=5)
        data = response.json()
        weather = {
            "temperature": float(data["main"]["temp"]),
            "humidity": int(data["main"]["humidity"]),
            "condition": data["weather"][0]["description"]
        }
        weather_cache[city] = {"fetched_at": now, "data": weather}
        return weather
    except Exception as e:
        print(f"⚠️ Weather fetch failed for {city}: {e}")
        return {"temperature": None, "humidity": None, "condition": "unknown"}

# Initialize Spark (unchanged)
spark = SparkSession.builder \
    .appName("FinTechStreamingPipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("✅ Spark Streaming session created")

# Corrected schema with TimestampType
schema = StructType() \
    .add("symbol", StringType()) \
    .add("price", FloatType()) \
    .add("volume", IntegerType()) \
    .add("timestamp", TimestampType()) \
    .add("exchange", StringType())

print("🌊 Starting streaming from Kafka...")

# Read from Kafka (unchanged)
streaming_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "test-topic") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load()

# Parse streaming data and convert timestamp
parsed_stream = streaming_df.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json("json_str", schema).alias("data")) \
    .select("data.*") \
    .withColumn("timestamp", to_timestamp(col("timestamp"))) \
    .withColumn("processing_time", lit(int(time.time())))

# Enrichment function (unchanged)
def enrich_and_store_stream(batch_df, batch_id):
    if batch_df.count() == 0:
        return
    
    print(f"🌊 Processing streaming batch {batch_id} with {batch_df.count()} records")
    
    exchanges = [row['exchange'] for row in batch_df.select("exchange").distinct().collect()]
    weather_data = {}
    
    for ex in exchanges:
        city = exchange_city_map.get(ex, "New York")
        print(f"🌤️ Fetching weather for {ex} ({city})")
        weather_data[ex] = fetch_weather_cached(city)
    
    enriched_df = batch_df
    for col_name in ["temperature", "humidity", "condition"]:
        enriched_df = enriched_df.withColumn(col_name, lit(None))
    
    for exchange, weather in weather_data.items():
        enriched_df = enriched_df \
            .withColumn("temperature", when(col("exchange") == exchange, lit(weather.get("temperature"))).otherwise(col("temperature"))) \
            .withColumn("humidity", when(col("exchange") == exchange, lit(weather.get("humidity"))).otherwise(col("humidity"))) \
            .withColumn("condition", when(col("exchange") == exchange, lit(weather.get("condition"))).otherwise(col("condition")))
    
    enriched_df = enriched_df \
        .withColumn("date", date_format(col("timestamp"), "yyyy-MM-dd"))
    
    # Handle schema changes with mergeSchema option
    enriched_df.write.format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .partitionBy("date") \
        .save("s3a://test-bucket/delta-tables/streaming_trades")
    
    print(f"✅ Batch {batch_id} written to streaming storage with date partitioning")
# Start the streaming query (unchanged)
streaming_query = parsed_stream.writeStream \
    .foreachBatch(enrich_and_store_stream) \
    .outputMode("append") \
    .trigger(processingTime='5 seconds') \
    .option("checkpointLocation", "s3a://test-bucket/checkpoints/streaming") \
    .start()

print("🌊 Streaming query started! Processing every 5 seconds...")
print("💡 Run the producer in another terminal to see real-time processing")
print("🌍 Weather data will be fetched for multiple cities based on exchanges")

try:
    streaming_query.awaitTermination()
except KeyboardInterrupt:
    print("\n🛑 Stopping streaming...")
    streaming_query.stop()
    print("✅ Streaming stopped")

✅ Spark Streaming session created
🌊 Starting streaming from Kafka...
🌊 Streaming query started! Processing every 5 seconds...
💡 Run the producer in another terminal to see real-time processing
🌍 Weather data will be fetched for multiple cities based on exchanges
🌊 Processing streaming batch 2 with 3 records
🌤️ Fetching weather for XLON (London)
🌤️ Fetching weather for XNYS (New York)
🌤️ Fetching weather for XNAS (New York)
✅ Batch 2 written to streaming storage with date partitioning
🌊 Processing streaming batch 3 with 14 records
🌤️ Fetching weather for XLON (London)
🌤️ Fetching weather for XNYS (New York)
🌤️ Fetching weather for XNAS (New York)
✅ Batch 3 written to streaming storage with date partitioning
🌊 Processing streaming batch 4 with 10 records
🌤️ Fetching weather for XNAS (New York)
🌤️ Fetching weather for XLON (London)
🌤️ Fetching weather for XNYS (New York)
✅ Batch 4 written to streaming storage with date partitioning
🌊 Processing streaming batch 5 with 10 records
🌤️ Fetchin

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt



🛑 Stopping streaming...
✅ Streaming stopped


In [12]:
# scripts/hourly_processor.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, avg, sum as spark_sum, count, max as spark_max, min as spark_min, lit
from datetime import datetime, timedelta
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_spark_session():
    return SparkSession.builder \
        .appName("FinTechHourlyProcessor") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.jars.packages", 
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
                "io.delta:delta-core_2.12:2.4.0,"
                "org.apache.hadoop:hadoop-aws:3.3.4,"
                "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
        .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
        .config("spark.hadoop.fs.s3a.access.key", "test") \
        .config("spark.hadoop.fs.s3a.secret.key", "test") \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .getOrCreate()

def run_hourly_processing():
    """
    Process the previous completed hour.
    E.g., if it's 10:05, process 09:00-09:59
    """
    spark = create_spark_session()
    spark.sparkContext.setLogLevel("WARN")
    
    try:
        # Calculate previous hour
        current_time = datetime.now()
        previous_hour = current_time.replace(minute=0, second=0, microsecond=0) #- timedelta(hours=1)
        target_hour = previous_hour.strftime("%Y-%m-%d %H")
        target_date = previous_hour.strftime("%Y-%m-%d")
        
        logger.info(f"🕐 Processing hourly data for: {target_hour}")
        
        # Read streaming data for the specific hour
        streaming_data = spark.read.format("delta") \
            .load("s3a://test-bucket/delta-tables/streaming_trades") \
            .filter(col("date") == target_date) \
            .filter(date_format(col("timestamp"), "yyyy-MM-dd HH") == target_hour)
        
        record_count = streaming_data.count()
        
        if record_count == 0:
            logger.warning(f"⚠️ No streaming data found for hour: {target_hour}")
            return
        
        logger.info(f"📈 Processing {record_count} records for hour: {target_hour}")
        
        # Create hourly aggregation
        hourly_summary = streaming_data \
            .withColumn("hour", lit(target_hour)) \
            .groupBy("symbol", "exchange", "hour", "date", "condition") \
            .agg(
                avg("price").alias("avg_price"),
                spark_min("price").alias("min_price"),
                spark_max("price").alias("max_price"),
                spark_sum("volume").alias("total_volume"),
                count("*").alias("trade_count"),
                avg("temperature").alias("avg_temperature"),
                avg("humidity").alias("avg_humidity")
            ) \
            .withColumn("batch_processed_at", lit(datetime.now().isoformat()))
        
        hourly_count = hourly_summary.count()
        logger.info(f"📊 Created {hourly_count} hourly summary records")
        
        if hourly_count > 0:
            # Use merge/upsert to replace only this specific hour
            hourly_summary.write.format("delta") \
                .mode("overwrite") \
                .option("replaceWhere", f"hour = '{target_hour}'") \
                .save("s3a://test-bucket/delta-tables/hourly_summaries")
            
            logger.info(f"✅ Hourly summary updated for {target_hour}")
        
        logger.info(f"🎉 Hourly processing completed for {target_hour}!")
        
    except Exception as e:
        logger.error(f"❌ Hourly processing failed: {str(e)}")
        raise
    finally:
        spark.stop()



In [ ]:
# scripts/daily_processor.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, avg, sum as spark_sum, count, max as spark_max, min as spark_min, lit
from datetime import datetime, timedelta
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_spark_session():
    return SparkSession.builder \
        .appName("FinTechDailyProcessor") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.jars.packages", 
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
                "io.delta:delta-core_2.12:2.4.0,"
                "org.apache.hadoop:hadoop-aws:3.3.4,"
                "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
        .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
        .config("spark.hadoop.fs.s3a.access.key", "test") \
        .config("spark.hadoop.fs.s3a.secret.key", "test") \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .getOrCreate()

def run_daily_processing():
    """
    Process yesterday's complete day for daily summaries.
    """
    spark = create_spark_session()
    spark.sparkContext.setLogLevel("WARN")
    
    try:
        # Calculate yesterday
        yesterday = (datetime.now() - timedelta(days=0)).strftime("%Y-%m-%d")
        
        logger.info(f"📅 Processing daily data for: {yesterday}")
        
        # Read all streaming data for yesterday
        streaming_data = spark.read.format("delta") \
            .load("s3a://test-bucket/delta-tables/streaming_trades") \
            .filter(col("date") == yesterday)
        
        record_count = streaming_data.count()
        
        if record_count == 0:
            logger.warning(f"⚠️ No streaming data found for date: {yesterday}")
            return
        
        logger.info(f"📈 Processing {record_count} records for date: {yesterday}")
        
        # Create daily aggregation
        daily_summary = streaming_data \
            .groupBy("symbol", "exchange", "date") \
            .agg(
                avg("price").alias("avg_price"),
                spark_min("price").alias("min_price"),
                spark_max("price").alias("max_price"),
                spark_sum("volume").alias("total_volume"),
                count("*").alias("trade_count"),
                avg("temperature").alias("avg_temperature"),
                avg("humidity").alias("avg_humidity")
            ) \
            .withColumn("batch_processed_at", lit(datetime.now().isoformat()))
        
        daily_count = daily_summary.count()
        logger.info(f"📅 Created {daily_count} daily summary records")
        
        if daily_count > 0:
            # Replace only yesterday's daily summary
            daily_summary.write.format("delta") \
                .mode("overwrite") \
                .option("replaceWhere", f"date = '{yesterday}'") \
                .save("s3a://test-bucket/delta-tables/daily_summaries")
            
            logger.info(f"✅ Daily summary updated for {yesterday}")
        
        logger.info(f"🎉 Daily processing completed for {yesterday}!")
        
    except Exception as e:
        logger.error(f"❌ Daily processing failed: {str(e)}")
        raise
    finally:
        spark.stop()


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, to_timestamp, date_format, avg, min as spark_min, max as spark_max, sum as spark_sum, count
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, TimestampType, LongType, DoubleType
from datetime import datetime, timedelta
import random

# Initialize Spark session (same as streaming_processor.py, without Kafka)
spark = SparkSession.builder \
    .appName("GenerateSampleDataAndAggregations") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# Define symbols and exchanges (match prod.py, add XTKS for streaming_processor.py)
symbols = {
    'XNAS': ['AAPL', 'MSFT', 'TSLA', 'GOOGL'],
    'XNYS': ['JPM', 'GE', 'XOM', 'BAC'],
    'XLON': ['BP', 'VODL', 'HSBA', 'AZN'],
    'XTKS': ['TM', 'SONY', 'HMC', 'MUFG']
}
exchanges = list(symbols.keys())
weather_conditions = ['clear', 'rain', 'cloudy', 'snow']

# Set date range: August 10–16, 2025 (7 days before August 17, 2025)
start_date = datetime(2025, 8, 10)

# Generate raw trade data for streaming_trades
data = []
for day in range(7):  # August 10–16, 2025
    for hour in range(24):
        timestamp = start_date + timedelta(days=day, hours=hour)
        for exchange in exchanges:
            for symbol in symbols[exchange]:
                for _ in range(random.randint(5, 10)):  # 5-10 trades per symbol per hour
                    trade_time = timestamp + timedelta(seconds=random.randint(0, 3600))
                    data.append((
                        symbol,
                        round(random.uniform(100, 300), 2),  # Match prod.py price range
                        random.randint(10, 1000),  # Match prod.py volume range
                        trade_time.isoformat(),  # String to match prod.py
                        exchange,
                        round(random.uniform(15, 30), 1),  # Match streaming_processor.py weather
                        random.randint(40, 80),
                        random.choice(weather_conditions),
                        trade_time.strftime("%Y-%m-%d"),
                        int(trade_time.timestamp())
                    ))

# Corrected schema with StructField
schema = StructType([
    StructField("symbol", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("volume", IntegerType(), True),
    StructField("timestamp", StringType(), True),  # String to match prod.py
    StructField("exchange", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("humidity", IntegerType(), True),
    StructField("condition", StringType(), True),
    StructField("date", StringType(), True),
    StructField("processing_time", LongType(), True)
])

raw_df = spark.createDataFrame(data, schema=schema) \
    .withColumn("timestamp", to_timestamp(col("timestamp")))  # Convert to TimestampType for streaming_processor.py

raw_df.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .partitionBy("date") \
    .save("s3a://test-bucket/delta-tables/streaming_trades")
print(f"✅ Wrote {raw_df.count()} records to streaming_trades")

# Apply hourly aggregations (match hourly_processor.py)
for day in range(7):
    target_date = (start_date + timedelta(days=day)).strftime("%Y-%m-%d")
    for hour in range(24):
        target_time = start_date + timedelta(days=day, hours=hour)
        target_hour = target_time.strftime("%Y-%m-%d %H")
        
        streaming_data = spark.read.format("delta") \
            .load("s3a://test-bucket/delta-tables/streaming_trades") \
            .filter(col("date") == target_date) \
            .filter(date_format(col("timestamp"), "yyyy-MM-dd HH") == target_hour)
        
        if streaming_data.count() > 0:
            hourly_summary = streaming_data \
                .withColumn("hour", lit(target_hour)) \
                .groupBy("symbol", "exchange", "hour", "date", "condition") \
                .agg(
                    avg("price").alias("avg_price"),
                    spark_min("price").alias("min_price"),
                    spark_max("price").alias("max_price"),
                    spark_sum("volume").alias("total_volume"),
                    count("*").alias("trade_count"),
                    avg("temperature").alias("avg_temperature"),
                    avg("humidity").alias("avg_humidity")
                ) \
                .withColumn("batch_processed_at", lit(datetime.now().isoformat()))
            
            hourly_summary.write.format("delta") \
                .mode("overwrite") \
                .option("replaceWhere", f"hour = '{target_hour}'") \
                .save("s3a://test-bucket/delta-tables/hourly_summaries")
            print(f"✅ Hourly summary for {target_hour} written")

print("✅ Hourly aggregations completed")

# Apply daily aggregations (match daily_processor.py)
for day in range(7):
    target_date = (start_date + timedelta(days=day)).strftime("%Y-%m-%d")
    
    streaming_data = spark.read.format("delta") \
        .load("s3a://test-bucket/delta-tables/streaming_trades") \
        .filter(col("date") == target_date)
    
    if streaming_data.count() > 0:
        daily_summary = streaming_data \
            .groupBy("symbol", "exchange", "date") \
            .agg(
                avg("price").alias("avg_price"),
                spark_min("price").alias("min_price"),
                spark_max("price").alias("max_price"),
                spark_sum("volume").alias("total_volume"),
                count("*").alias("trade_count"),
                avg("temperature").alias("avg_temperature"),
                avg("humidity").alias("avg_humidity")
            ) \
            .withColumn("batch_processed_at", lit(datetime.now().isoformat()))
        
        daily_summary.write.format("delta") \
            .mode("overwrite") \
            .option("replaceWhere", f"date = '{target_date}'") \
            .save("s3a://test-bucket/delta-tables/daily_summaries")
        print(f"✅ Daily summary for {target_date} written")

print("✅ Daily aggregations completed")

spark.stop()

✅ Wrote 20321 records to streaming_trades
✅ Hourly summary for 2025-08-10 00 written
✅ Hourly summary for 2025-08-10 01 written
✅ Hourly summary for 2025-08-10 02 written
✅ Hourly summary for 2025-08-10 03 written
✅ Hourly summary for 2025-08-10 04 written
✅ Hourly summary for 2025-08-10 05 written
✅ Hourly summary for 2025-08-10 06 written
✅ Hourly summary for 2025-08-10 07 written
✅ Hourly summary for 2025-08-10 08 written
✅ Hourly summary for 2025-08-10 09 written
✅ Hourly summary for 2025-08-10 10 written
✅ Hourly summary for 2025-08-10 11 written
✅ Hourly summary for 2025-08-10 12 written
✅ Hourly summary for 2025-08-10 13 written
✅ Hourly summary for 2025-08-10 14 written
✅ Hourly summary for 2025-08-10 15 written
✅ Hourly summary for 2025-08-10 16 written
✅ Hourly summary for 2025-08-10 17 written
✅ Hourly summary for 2025-08-10 18 written
✅ Hourly summary for 2025-08-10 19 written
✅ Hourly summary for 2025-08-10 20 written
✅ Hourly summary for 2025-08-10 21 written
✅ Hourly sum